
Отобразите все записи из таблицы company по компаниям, которые закрылись.

In [1]:
query = '''
SELECT *
FROM company

WHERE status = 'closed'
'''

Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total.

In [2]:
query = '''
SELECT funding_total
FROM company
WHERE category_code = 'news' AND country_code = 'USA'
ORDER BY funding_total DESC
'''

Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [3]:
query = '''
SELECT SUM(price_amount)

FROM acquisition 

WHERE EXTRACT(YEAR FROM CAST(acquired_at AS date)) BETWEEN 2011 AND 2013 AND term_code = 'cash'
'''

Отобразите имя, фамилию и названия аккаунтов людей в поле network_username, у которых названия аккаунтов начинаются на 'Silver'.

In [4]:
query = '''
SELECT first_name,last_name,network_username
FROM people

WHERE network_username LIKE 'Silver%'
'''


Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле network_username содержат подстроку 'money', а фамилия начинается на 'K'.

In [5]:
query = '''
SELECT *
FROM people
WHERE network_username LIKE '%money%' AND last_name LIKE 'K%'
'''

Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [6]:
query = '''
SELECT country_code,SUM(funding_total)
FROM company 

GROUP BY country_code 
ORDER BY SUM(funding_total) DESC
'''

Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [14]:
query = '''
WITH InvestmentStats AS (
    SELECT
        funded_at,
        MIN(raised_amount) AS min_raised_amount,
        MAX(raised_amount) AS max_raised_amount
    FROM funding_round
    GROUP BY funded_at
)

SELECT
    funded_at,
    min_raised_amount,
    max_raised_amount
FROM InvestmentStats
WHERE min_raised_amount != 0
  AND min_raised_amount != max_raised_amount;

'''

Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.

In [15]:
query = '''
SELECT *,
      CASE
      WHEN invested_companies >= 100 THEN 'high_activity'
      WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
      WHEN invested_companies  < 20 THEN 'low_activity'
      END 
FROM fund
'''

Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [16]:
query = '''
SELECT 
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds))
FROM fund

GROUP BY activity

ORDER BY ROUND(AVG(investment_rounds)) ASC
'''

Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [19]:
query = '''
WITH FilteredFunds AS (
    SELECT 
        country_code,
        invested_companies
    FROM 
        fund
    WHERE 
        DATE_PART('year', founded_at) BETWEEN 2010 AND 2012
)

SELECT 
    country_code,
    MIN(invested_companies) AS min_in,
    MAX(invested_companies) AS max_in,
    AVG(invested_companies) AS avg_in
FROM 
    FilteredFunds
GROUP BY 
    country_code
HAVING 
    MIN(invested_companies) != 0
ORDER BY 
    avg_in DESC, 
    country_code ASC
LIMIT 
    10;

'''

Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [20]:
query = '''
SELECT first_name,
       last_name,
       e.instituition
FROM people AS p

LEFT JOIN education AS e ON p.id=e.person_id
'''


Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [21]:
query = '''
SELECT 
    c.name, 
    COUNT(DISTINCT e.instituition) AS unique_institutions
FROM 
    company AS c
INNER JOIN 
    people AS p ON c.id = p.company_id
LEFT JOIN 
    education AS e ON p.id = e.person_id
GROUP BY 
    c.name
ORDER BY 
    unique_institutions DESC
LIMIT 
    5;

'''


Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [12]:
query = '''
SELECT DISTINCT c.name 

FROM ( 
SELECT *
FROM funding_round
WHERE is_first_round = 1 AND is_last_round = 1 ) AS qw
 
INNER JOIN company AS c ON qw.company_id=c.id

WHERE status = 'closed'
'''

Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [29]:
query = '''
WITH SelectedCompanies AS 
(SELECT DISTINCT c.id

FROM ( 
SELECT *
FROM funding_round
WHERE is_first_round = 1 AND is_last_round = 1 ) AS qw
 
INNER JOIN company AS c ON qw.company_id=c.id

WHERE status = 'closed'
)

SELECT DISTINCT p.id
FROM people AS p

INNER JOIN SelectedCompanies AS Sc ON p.company_id=Sc.id
'''

Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник

In [30]:
query = '''
WITH SelectedCompanies AS 
(SELECT DISTINCT c.id

FROM ( 
SELECT *
FROM funding_round
WHERE is_first_round = 1 AND is_last_round = 1 ) AS qw
 
INNER JOIN company AS c ON qw.company_id=c.id

WHERE status = 'closed'
)

SELECT DISTINCT p.id,e.instituition
FROM people AS p

INNER JOIN SelectedCompanies AS Sc ON p.company_id=Sc.id
INNER JOIN education AS e ON p.id = e.person_id

'''

Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [31]:
query = '''
WITH SelectedCompanies AS 
(SELECT DISTINCT c.id

FROM ( 
SELECT *
FROM funding_round
WHERE is_first_round = 1 AND is_last_round = 1 ) AS qw
 
INNER JOIN company AS c ON qw.company_id=c.id

WHERE status = 'closed'
)

SELECT DISTINCT p.id,COUNT(e.instituition)
FROM people AS p

INNER JOIN SelectedCompanies AS Sc ON p.company_id=Sc.id
INNER JOIN education AS e ON p.id = e.person_id

group BY p.id
'''

Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [24]:
query = '''
WITH SelectedCompanies AS 
(SELECT DISTINCT c.id

FROM ( 
SELECT *
FROM funding_round
WHERE is_first_round = 1 AND is_last_round = 1 ) AS qw
 
INNER JOIN company AS c ON qw.company_id=c.id

WHERE status = 'closed'
)

SELECT AVG(education_count)
FROM (
SELECT DISTINCT p.id,COUNT(e.instituition) AS education_count
FROM people AS p

INNER JOIN SelectedCompanies AS Sc ON p.company_id=Sc.id
INNER JOIN education AS e ON p.id = e.person_id

group BY p.id) AS qwe

'''

In [ ]:
Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Socialnet.

In [25]:
query = '''
WITH SelectedCompanies AS 
(SELECT DISTINCT c.id

FROM ( 
SELECT *
FROM funding_round
) AS qw
 
INNER JOIN company AS c ON qw.company_id=c.id

WHERE c.name = 'Socialnet'
)

SELECT AVG(education_count)
FROM (
SELECT DISTINCT p.id,COUNT(e.instituition) AS education_count
FROM people AS p

INNER JOIN SelectedCompanies AS Sc ON p.company_id=Sc.id
INNER JOIN education AS e ON p.id = e.person_id

group BY p.id) AS qwe

'''

Составьте таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [26]:
query = '''

SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM fund AS f


INNER JOIN investment AS i ON f.id=i.fund_id
INNER JOIN funding_round AS fr ON i.funding_round_id=fr.id
INNER JOIN company AS c ON fr.company_id = c.id

WHERE c.milestones > 6 AND EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN 2012 AND 2013

GROUP BY name_of_fund,name_of_company,amount
'''


Выгрузите таблицу, в которой будут такие поля:
название компании-покупателя;
сумма сделки;
название компании, которую купили;
сумма инвестиций, вложенных в купленную компанию;
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [27]:
query = '''
SELECT c.name AS buyers_company,
       a.price_amount AS sum_transactions,
       cc.name AS bought_company,
       cc.funding_total AS sum_invest,
       ROUND(a.price_amount/cc.funding_total) AS dolia

FROM acquisition AS a

INNER JOIN company AS c ON a.acquiring_company_id = c.id

INNER JOIN company AS cc ON a.acquired_company_id = cc.id


WHERE a.price_amount != 0 AND cc.funding_total != 0

ORDER BY sum_transactions DESC, bought_company ASC
LIMIT 10
'''

Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [32]:
query = '''
SELECT c.name,
       EXTRACT(MONTH FROM CAST(funded_at AS date))
FROM company AS c

INNER JOIN funding_round AS fr ON c.id=fr.company_id

WHERE category_code = 'social' AND fr.raised_amount != 0 AND EXTRACT(YEAR FROM CAST(funded_at AS date)) BETWEEN 2010 AND 2013


'''

Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
количество уникальных названий фондов из США, которые инвестировали в этом месяце;
количество компаний, купленных за этот месяц;
общая сумма сделок по покупкам в этом месяце.

In [33]:
query = '''
WITH
 
acq_c AS (SELECT EXTRACT(MONTH FROM CAST(acquired_at AS DATE)) AS MONTH,
                 COUNT(acquired_company_id) AS count_comp,
                 SUM(price_amount) AS sum_amount
          FROM acquisition
          WHERE EXTRACT(YEAR FROM acquired_at) BETWEEN 2010 AND 2013
          GROUP BY MONTH),
 
fr_uniq AS (SELECT EXTRACT(MONTH FROM CAST(funded_at AS DATE)) AS MONTH,
                   COUNT(DISTINCT f.name) AS name_fund
            FROM funding_round AS fr
            LEFT JOIN investment AS i ON i.funding_round_id = fr.id
            LEFT JOIN fund AS f ON f.id = i.fund_id
            WHERE EXTRACT(YEAR FROM funded_at) BETWEEN 2010 AND 2013
                  AND f.country_code = 'USA'
            GROUP BY MONTH)
            
SELECT fr_uniq.MONTH,
       fr_uniq.name_fund,
       acq_c.count_comp,
       acq_c.sum_amount
FROM fr_uniq
JOIN acq_c ON acq_c.MONTH = fr_uniq.MONTH;
'''

Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [36]:
query = '''
WITH
 
avg2011 AS (SELECT country_code,
                   AVG(funding_total) AS avg_total_2011
            FROM company
            WHERE EXTRACT(YEAR FROM CAST(founded_at AS TIMESTAMP)) = 2011
            GROUP BY country_code),
            
avg2012 AS (SELECT country_code,
                   AVG(funding_total) AS avg_total_2012
            FROM company
            WHERE EXTRACT(YEAR FROM CAST(founded_at AS TIMESTAMP)) = 2012
            GROUP BY country_code),
 
avg2013 AS (SELECT country_code,
                   AVG(funding_total) AS avg_total_2013
            FROM company
            WHERE EXTRACT(YEAR FROM CAST(founded_at AS TIMESTAMP)) = 2013
            GROUP BY country_code)
                
SELECT avg2011.country_code AS country_code,
       avg2011.avg_total_2011,
       avg2012.avg_total_2012,
       avg2013.avg_total_2013
FROM avg2011
INNER JOIN avg2012 ON avg2011.country_code=avg2012.country_code
INNER JOIN avg2013 ON avg2012.country_code=avg2013.country_code
ORDER BY avg2011.avg_total_2011 DESC;
'''